# 卷积神经网络对图片进行分类。kaggle

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import os,time
import numpy as np
import pandas as pd
import sklearn
import sys
import tensorflow as tf
from tensorflow import keras

print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)


sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.2.0
numpy 1.18.1
pandas 1.0.1
sklearn 0.22.2.post1
tensorflow 2.1.0
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf


In [2]:
tf.debugging.set_log_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0],'GPU')# 设定只第一个GPU是可见的
tf.config.experimental.set_virtual_device_configuration(# 给GPU做逻辑切分
                gpus[0],# 制定使用哪个gpu
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096),
                tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096),
                tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096),
                tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096),]
        )
print(len(gpus))
logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print(len(logical_gpus))


1
4


In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
class_names = [
        'airplane',
        'automobile',
        'bird' ,
        'cat' ,
        'deer' ,
        'dog' ,
        'frog' ,
        'horse' ,
        'ship' ,
        'truck',
        ]
train_labels_file = 'cifar10/trainLabels.csv'
test_csv_file = 'cifar10/sampleSubmission.csv'
train_folder = 'cifar10/train/'
test_folder = 'cifar10/test/'

def parse_csv_file(filepath, folder):
    """Parses csv files into (filename(full_path), label) format"""
    results = []
    with open(filepath, 'r') as f:
        lines = f.readlines()
    for line in lines:
        image_id, label_str = line.strip('\n').split(',')
        image_full_path = os.path.join(folder, image_id + '.png')
        results.append((image_full_path,label_str))
    return results
train_labels_info = parse_csv_file(train_labels_file, train_folder)
test_csv_info = parse_csv_file(test_csv_file,test_folder)

# import pprint
# pptint.pprint(train_labels_info[:5])
# pprint.pprint(test_csv_info[:5])
# print(len(train_labels_info), len(test_csv_info))

In [6]:
# train_df = pd.Dataframe(train_labels_info)
train_df = pd.DataFrame(train_labels_info[:45000])
valid_df = pd.DataFrame(train_labels_info[45000:])
test_df = pd.DataFrame(test_csv_info)[1:]

train_df.columns = ['filepath', 'class']
valid_df.columns = ['filepath', 'class']
test_df.columns = ['filepath', 'class']

print(train_df.head())
print(train_df.shape)
print(test_df.head())
print(test_df.shape)
print(valid_df.head())
print(valid_df.shape)


               filepath  class
0  cifar10/train/id.png  label
1   cifar10/train/1.png   frog
2   cifar10/train/2.png  truck
3   cifar10/train/3.png  truck
4   cifar10/train/4.png   deer
(45000, 2)
             filepath class
1  cifar10/test/1.png   cat
2  cifar10/test/2.png   cat
3  cifar10/test/3.png   cat
4  cifar10/test/4.png   cat
5  cifar10/test/5.png   cat
(300000, 2)
                  filepath       class
0  cifar10/train/45000.png        frog
1  cifar10/train/45001.png       horse
2  cifar10/train/45002.png  automobile
3  cifar10/train/45003.png        deer
4  cifar10/train/45004.png  automobile
(5001, 2)


In [7]:
height = 32# 定义将图片缩放到同样大小图片大小
width = 32 
channels = 3# 彩色图片，3通道
batch_size = 32 # 生产的图片多少张为一组
num_classes = 10# 类别的个数
# ImageDataGenerator读取图片并作数据增强
# 1、初始化一个generator
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,# 将图片随机旋转角度，40指0-40之间随机选一个数
    width_shift_range= 0.2,# 对数据进行位移小于1代表最大比例，大于1代表最大像素值
    height_shift_range =0.2,
    shear_range=0.2,# 剪切强度
    zoom_range=0.2,# 缩放强度
    horizontal_flip= True,# 随机水平翻转
    fill_mode='nearest',# 如果填充像素，采用的方法
)
# 2、读取图片
train_generator = train_datagen.flow_from_dataframe(train_df,
#                                                    directory = './',
                                                   x_col='filepath',
                                                   y_col = 'class',
                                                   classes= class_names,
                                                   target_size = (height, width),
                                                   batch_size = batch_size,
                                                   seed = 7,
                                                   shuffle = True,
                                                   class_mode = 'sparse'
                                                   )
valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
#  验证集一般不对图像处理，只要缩放值就可以
valid_generator = valid_datagen.flow_from_dataframe(valid_df,
#                                                    directory = './',
                                                   x_col = 'filepath',
                                                   y_col = 'class',
                                                   classes = class_names,
                                                   target_size=(height,width),
                                                   batch_size=batch_size,
                                                   seed = 7,
                                                   shuffle= False,
                                                   class_mode = 'sparse')

train_num = train_generator.samples#查看数据数量
valid_num = valid_generator.samples
print(train_num,valid_num)

Found 44999 validated image filenames belonging to 10 classes.
Found 5001 validated image filenames belonging to 10 classes.
44999 5001


/home/zo1zu22/env/tf2_py3/lib/python3.6/site-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 1 invalid image filename(s) in x_col="filepath". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [8]:
for i in range(2):
    x,y = train_generator.next()
    print(x.shape,y.shape)
    print(y)

(32, 32, 32, 3) (32,)
[1. 4. 4. 9. 9. 8. 6. 8. 3. 7. 4. 2. 1. 1. 8. 1. 6. 2. 8. 0. 1. 4. 3. 5.
 6. 6. 7. 4. 9. 1. 0. 2.]
(32, 32, 32, 3) (32,)
[9. 0. 2. 0. 9. 7. 5. 6. 5. 6. 6. 0. 0. 3. 0. 5. 0. 7. 8. 5. 9. 4. 1. 4.
 5. 2. 3. 6. 1. 5. 7. 0.]


In [8]:
#实现多GPU分布式
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    #把层次和compile都放在mirroredsttategy下
    model = keras.models.Sequential([
        keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',
                           activation='selu',input_shape=[width,height,channels]),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',
                               activation='selu'),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPool2D(pool_size=2),#MaxPool，Avgpool

        keras.layers.Conv2D(filters=256,kernel_size=3,padding='same',
                           activation='selu'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(filters=256,kernel_size=3,padding='same',
                           activation='selu'),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPool2D(pool_size=2),

        keras.layers.Conv2D(filters=512,kernel_size=3,padding='same',
                           activation='selu'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(filters=512,kernel_size=3,padding='same',
                           activation='selu'),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPool2D(pool_size=2),
        keras.layers.Flatten(),
        keras.layers.Dense(128,activation='selu'),
        keras.layers.Dense(num_classes,activation='softmax')
    ])
    #由于class_mode是sparse，所以使用sparse_categorical_crossentropy
    model.compile(loss ='sparse_categorical_crossentropy',
                 optimizer='adam',metrics=['accuracy'])
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhos

In [9]:
epochs = 100
# 数据由generator生成，所以用fit_generator
history = model.fit(train_generator,
                             steps_per_epoch=train_num//batch_size,
                              epochs= epochs,
                          validation_data = valid_generator,
                              validation_steps = valid_num//batch_size
                             )

  ...
    to  
  ['...']
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
  ...
    to  
  ['...']
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DatasetCardinality in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RebatchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AutoShardDa

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:3
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:3
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:3
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:3
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:3
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:3
Executing op VarHandleOp in device /job:

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localho

1406/1406 [==============================] - 47s 34ms/step - loss: 1.2467 - accuracy: 0.5556 - val_loss: 1.1322 - val_accuracy: 0.6086
Epoch 5/100
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:3
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:3
1405/1406 [============================>.] - ETA: 0s - loss: 1.1254 - accuracy: 0.6004Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:0
Execut

...

Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:3
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:3
1405/1406 [============================>.] - ETA: 0s - loss: 0.2331 - accuracy: 0.9205Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:3
Executing op PrefetchDataset in device /job:localhost/replica:0/task:

Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:3
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:3
1406/1406 [==============================] - 47s 33ms/step - loss: 0.2101 - accuracy: 0.9281 - val_loss: 0.4661 - val_accuracy: 0.8878
Epoch 78/100
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:2
Executi

Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:3
1406/1406 [==============================] - 47s 33ms/step - loss: 0.1990 - accuracy: 0.9326 - val_loss: 0.5531 - val_accuracy: 0.8798
Epoch 83/100
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:3
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:3
1404/1406 [============================>.] - ETA: 0s - loss: 0.1994 - accuracy: 0.9315Execut

Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:3
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:3
1405/1406 [============================>.] - ETA: 0s - loss: 0.1878 - accuracy: 0.9364Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op PrefetchDataset in device /job:localhost/replica:0/task:

Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:3
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:3
1404/1406 [============================>.] - ETA: 0s - loss: 0.1810 - accuracy: 0.9398Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:3
Executing op PrefetchDataset in device /job:localhost/replica:0/task:

In [ ]:
print(history.history.keys())

In [10]:
model.save('cifar10/cifar10_model_0320.h5')

In [9]:
# 重新创建完全相同的模型，包括其权重和优化程序
new_model = keras.models.load_model('cifar10/cifar10_model_0320.h5')

# 显示网络结构
new_model.summary()

Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/tas

In [5]:
data={}
def plot_learning_curves(history,label, epochs, min_value, max_value):
    data[label] = history.history[label]
    data['val_'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize = (8,5))
    plt.grid(True)
#     plt.axis([0,epochs],[min_value,max_value])
    
plot_learning_curves(history,'accuracy',epochs,0,1)
plot_learning_curves(history,'loss',epochs,1,2.5)

NameError: name 'history' is not defined

In [10]:
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
#  验证集一般不对图像处理，只要缩放值就可以
test_generator = valid_datagen.flow_from_dataframe(test_df,
                                                #    directory = './',
                                                   x_col = 'filepath',
                                                   y_col = 'class',
                                                   classes = class_names,
                                                   target_size=(height,width),
                                                   batch_size=batch_size,
                                                   seed = 7,
                                                   shuffle= False,
                                                   class_mode = 'sparse')
test_num = test_generator.samples
print(test_num)

/home/zo1zu22/env/tf2_py3/lib/python3.6/site-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 36229 invalid image filename(s) in x_col="filepath". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 263771 validated image filenames belonging to 10 classes.
263771


In [ ]:
test_predict = new_model.predict_generator(test_generator,
                                            workers = 10,# 并行处理量
                                            use_multiprocessing = True# 多进程，false多线程
                                                 )

Instructions for updating:
Please use Model.predict, which supports generators.
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_distributed_function_3102 in device /job:localhost/replica:0/task:0/device:GPU:0


In [ ]:
print（test_predict.shape）

In [ ]:
print(test_predict[0:5])

In [ ]:
test_predict_class_indices = np.argmax(test_predict, axis = 1)
print(test_predict_class_indices[:5])

In [ ]:
test_predict_class = [class_names[index] for index in test_predict_class_indices]
print(test_predict_class[:5])

In [ ]:
def generate_submissions(filename, predict_class):
    with open(filename, 'w') as f:
        f.write('id,label\n')
        for i in range(len(predict_class)):
            f.write('%d,%s\n' %(i+1, predict_class[i]))
            
output_file = 'data/submission.csv'
generate_submissions(output_file, test_predict_class)

In [ ]:
# https://www.kaggle.com/c/cifar-10/data
# late submossion上传submission.csv文件
# 描述文件，写主要参数，
# eg.
# 'train 45000 20 epichs, basic model with only conv + bn + pooling'